In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from recomm.features import index_single_feature
from recomm.classifier import ClassifierNN, ClassifierDNN

In [2]:
init_notebook_mode(connected=True)

# Introduction

In this document, we are going to check whether the neural network is able to split a series number between 0 and 1.

In [3]:
discriminate_level = 5
classifier = []
data = []
for level in range(2, discriminate_level + 1):
    features = np.random.rand(50000)
    labels = features
    features = pd.DataFrame(features)
    labels = pd.get_dummies(np.ceil(level * labels))
    train_features = features.iloc[:40000].values
    train_labels = labels.iloc[:40000].values
    test_features = features.iloc[40000:].values
    test_labels = labels.iloc[40000:].values
    classifier.append(ClassifierNN(train_features, train_labels)\
                        .build_network()\
                        .set_objective(method="l2_loss")\
                        .optimize()\
                        .estimate(batch_size=1000, learning_rate=1e-1)\
                        .predict(test_features)\
                        .activate_label()\
                        .get_accuracy(test_labels))
    data.append(go.Scatter(x=np.arange(1000),
                   y=np.log(np.array(classifier[-1].loss)),
                   mode="lines",
                   name="level: {}".format(level)))

In [4]:
features.head()

,0
0,0.656742
1,0.046353
2,0.476321
3,0.480099
4,0.159748


In [5]:
labels.head()

,1.0,2.0,3.0,4.0,5.0
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,1,0,0,0,0


In [6]:
layout = go.Layout(
    title='Learning Rate: 1e-1',
    xaxis=dict(
        title='iteration steps',
    ),
    yaxis=dict(
        title='L2 Loss (Log Scale)'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Unforunately, it is a challenge to discriminate levels by adopting neural network, and the fidelity drops linearly with respect to the level.

In [7]:
data = [go.Scatter(x=np.arange(2, discriminate_level + 1),
                   y=[item.accuracy for item in classifier],
                   mode="lines",
                   name="batch_size: 50"),]
layout = go.Layout(
    title='Learning Rate: 1e-1',
    xaxis=dict(
        title='Level for Discrimination',
    ),
    yaxis=dict(
        title='L2 Fidelity'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)